# **Chat Bot code workflow**

In [ ]:
#libraries and Configuration

import os
import asyncio
from typing import List, Dict, Any

from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings # Import VertexAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_vertexai import VertexAI, VertexVectorStore # Import VertexVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader # Keep for reference, but we'll use GCS for scraped data
from google.cloud import storage
from langchain_openai import ChatOpenAI # Import ChatOpenAI for GPT-4


# --- GCP Configuration ---
PROJECT_ID = " "
REGION = " "
GCS_BUCKET_NAME = " "
# The GCS folder where scraped HTML/text files are stored
GCS_SCRAPED_DATA_PREFIX = " "
# Vertex AI Search App ID for your data store
VERTEX_SEARCH_DATASTORE_ID = "your-vertex-ai-search-datastore-id"
# Vertex AI Vector Search Index ID and Endpoint ID
VERTEX_VECTOR_SEARCH_INDEX_ID = "your-vertex-vector-search-index-id"
VERTEX_VECTOR_SEARCH_ENDPOINT_ID = "your-vertex-vector-search-endpoint-id"

# 1. **Webscraping**

In [ ]:
from google.cloud import storage
import requests
from bs4 import BeautifulSoup
import os


PROJECT_ID = " "
GCS_BUCKET_NAME = " "

# Authenticate implicitly if running on a GCP service with service account permissions
# Otherwise, you might need to set GOOGLE_APPLICATION_CREDENTIALS environment variable
# pointing to your service account key file.
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/key.json"

def scrape_website_and_upload_to_gcs(url, gcs_path_prefix="scraped_data/"):
    """
    Scrapes a given URL, extracts text, and uploads it to Google Cloud Storage.

    Args:
        url (str): The URL to scrape.
        gcs_path_prefix (str): The GCS folder path where the file will be stored.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')
        # Example: Extract all text from paragraphs. Adjust as per your scraping needs.
        scraped_text = ' '.join([p.get_text() for p in soup.find_all('p')])

        if not scraped_text:
            print(f"No significant text found for {url}. Skipping upload.")
            return

        # Define the GCS file name. You might want to sanitize the URL for this.
        file_name = url.split("//")[-1].replace("/", "_").replace(".", "_") + ".txt"
        destination_blob_name = os.path.join(gcs_path_prefix, file_name)

        storage_client = storage.Client(project=PROJECT_ID)
        bucket = storage_client.bucket(GCS_BUCKET_NAME)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_string(scraped_text, content_type='text/plain')
        print(f"Uploaded scraped data from {url} to gs://{GCS_BUCKET_NAME}/{destination_blob_name}")

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    urls_to_scrape = [
        "https://www.example.com/page1",
        "https://www.example.org/another-page",
        # Add more URLs as needed
    ]

    for url in urls_to_scrape:
        scrape_website_and_upload_to_gcs(url)

# **2. Splitting**, **Chunking**, **Embeddings**

In [ ]:
# 1. Text Splitter for RAG
# Chunk size and overlap are crucial for RAG performance
# Adjust these based on the nature of your e-commerce product descriptions.
# For detailed product info, larger chunks might be needed.
# For short Q&A, smaller chunks.
CHUNK_SIZE = 1000 # Characters per chunk
CHUNK_OVERLAP = 200 # Overlap between chunks to maintain context

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)

# 2. Embeddings (Vertex AI Embeddings)
# Using Vertex AI's text-embedding-004 model
embeddings = VertexAIEmbeddings(model_name="text-embedding-004", project=PROJECT_ID)

# 3. LLM: GPT-4 via Langchain's OpenAI integration
# For GPT-4, ensure your OPENAI_API_KEY is set in your environment variables.
llm = ChatOpenAI(model_name="gpt-4", temperature=0) # Using GPT-4 model



# **3.Search and Retrieval**

In [ ]:
# --- Google Search Tool (for when RAG doesn't have the answer or for general queries) ---
# We'll use Vertex AI Search for this.
# Ensure the Vertex AI Search Data Store is configured for web search or specific data.
# This requires `google-cloud-discoveryengine` library.
from google.cloud import discoveryengine_v1beta as discoveryengine

class VertexAISearchTool:
    def __init__(self, project_id: str, location: str, data_store_id: str):
        self.project_id = project_id
        self.location = location
        self.data_store_id = data_store_id
        self.client = discoveryengine.SearchServiceClient()
        self.serving_config = self.client.serving_config_path(
            project=self.project_id,
            location=self.location,
            data_store=self.data_store_id,
            serving_config="default_config",
        )

    def search(self, query: str) -> str:
        try:
            request = discoveryengine.SearchRequest(
                serving_config=self.serving_config,
                query=query,
                page_size=3, # Retrieve top 3 results
            )
            response = self.client.search(request)

            results = []
            for result in response.results:
                if result.document and result.document.derived_struct_data:
                    snippet = result.document.derived_struct_data.get("snippet", "No snippet available.")
                    url = result.document.derived_struct_data.get("url", "No URL available.")
                    results.append(f"Source: {url}\nSnippet: {snippet}")
            if results:
                return "\n\n".join(results)
            else:
                return "No relevant information found using Google Search."
        except Exception as e:
            return f"Error performing Vertex AI Search: {e}"

vertex_ai_search_tool = VertexAISearchTool(
    project_id=PROJECT_ID,
    location=REGION,
    data_store_id=VERTEX_SEARCH_DATASTORE_ID
)


# --- Request Body for Chatbot API ---
class ChatRequest(BaseModel):
    query: str


# --- Retrieval Setup ---
# You'll use Vertex AI Vector Search (Matching Engine) as your vector database.
# For this, you need to ensure your index is created and deployed.

# --- Initialize Vertex AI Vector Store ---
# In a real-time production scenario, your Vertex AI Vector Search index
# would be pre-populated with data. This code connects to that existing index.
try:
    vectorstore = VertexVectorStore(
        project_id=PROJECT_ID,
        region=REGION,
        index_id=VERTEX_VECTOR_SEARCH_INDEX_ID,
        endpoint_id=VERTEX_VECTOR_SEARCH_ENDPOINT_ID,
        embedding=embeddings,
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 relevant chunks
    print("Successfully connected to Vertex AI Vector Search and initialized retriever.")
except Exception as e:
    print(f"Failed to connect to Vertex AI Vector Search: {e}")
    print("Ensure your Vertex AI Vector Search index is created, deployed, and the environment variables/permissions are correct.")
    retriever = None # Ensure retriever is None if connection fails


# Fallback for retriever if connection to Vertex AI Vector Search fails
if retriever is None:
    print("Warning: Retriever not initialized. Chatbot will not be able to answer context-based questions.")
    print("Falling back to only Google Search for all queries if RAG is not available.")
    # You might want to explicitly set a simpler LLM chain here without retrieval
    # or handle this in the /chat endpoint directly.

# **4.Prompt**

In [ ]:
# 4. Prompt Template for the Chatbot
# This template guides the LLM on how to answer questions given retrieved context.
# Key considerations:
# - Context: Clearly tell the LLM that the provided context is product information.
# - Tone: Helpful, informative, slightly enthusiastic (e.g., "We're here to help you!").
# - Format: Encourage structured answers (e.g., bullet points for features).
CHATBOT_PROMPT = PromptTemplate(
    template="""You are a friendly and knowledgeable AI assistant for an e-commerce website.
Your goal is to help customers find information about our products and make informed purchasing decisions.
Use the following pieces of product information (context) to answer the user's question.
If the answer is not found in the context, politely state that you don't have information on that specific topic
and suggest contacting customer support for further assistance or browsing the website.
Do not make up information.

Context:
{context}

Question: {question}

Answer:""",
    input_variables=["context", "question"],
)


# **5.LLM Orchestration**

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # Now using the ChatOpenAI instance for GPT-4
    chain_type="stuff", # 'stuff' combines all documents into one prompt. 'map_reduce' etc. for more documents.
    retriever=retriever, # This is the Vertex AI Vector Search retriever in a production setup
    return_source_documents=True,
    chain_type_kwargs={"prompt": CHATBOT_PROMPT}
)

# **FAST API**

In [ ]:
# --- FastAPI App Setup ---
app = FastAPI(
    title="E-commerce Product Chatbot",
    description="An AI-powered chatbot for e-commerce product inquiries, leveraging Google Cloud's Vertex AI.",
    version="1.0.0"
)

# Templates for serving HTML (if you want a simple frontend with FastAPI)
templates = Jinja2Templates(directory="templates")

# --- Initialize Google Cloud Storage Client ---
storage_client = storage.Client(project=PROJECT_ID)
gcs_bucket = storage_client.bucket(GCS_BUCKET_NAME)


# --- FastAPI Endpoints ---

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    """
    Serves a simple HTML page for the chatbot interface.
    """
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/chat")
async def chat_endpoint(chat_request: ChatRequest):
    """
    Handles user queries, retrieves information, and generates responses.
    """
    query = chat_request.query

    # First, try to answer using RAG (Vector Database)
    rag_response = None
    if retriever:
        try:
            rag_result = qa_chain.invoke({"query": query})
            rag_answer = rag_result.get("result")
            source_docs = rag_result.get("source_documents", [])

            # Check if the RAG answer indicates it couldn't find relevant info
            # This is based on the prompt template's instruction.
            if "not found in the context" in rag_answer.lower() or "politely state that you don't have information" in rag_answer.lower():
                rag_response = None # RAG couldn't answer definitively
            else:
                rag_response = rag_answer
                sources_info = "\nSources:\n" + "\n".join([doc.metadata.get("source", "N/A") for doc in source_docs])
                rag_response += sources_info

        except Exception as e:
            print(f"Error during RAG retrieval: {e}")
            rag_response = None

    # If RAG didn't provide a good answer, use the Google Search tool
    final_answer = ""
    if rag_response:
        final_answer = rag_response
    else:
        print(f"RAG did not provide a definitive answer for: {query}. Falling back to Google Search.")
        search_results = vertex_ai_search_tool.search(query)

        # Combine search results with a prompt for the LLM to synthesize an answer
        # This creates a "tool-augmented" RAG approach
        if "No relevant information found" in search_results or "Error performing Vertex AI Search" in search_results:
             final_answer = ("I couldn't find a direct answer to your question based on our product information "
                             "or general search. Please try rephrasing your question or contact our customer support for assistance.")
        else:
            # Use the LLM to synthesize an answer from the search results
            synthesis_prompt = PromptTemplate(
                template="""Based on the following search results, answer the user's question.
If the search results don't contain enough information, state that.
Search Results:
{search_results}

Question: {question}

Answer:""",
                input_variables=["search_results", "question"],
            )
            synthesis_chain = synthesis_prompt | llm
            try:
                final_answer = synthesis_chain.invoke({"search_results": search_results, "question": query})
                final_answer += f"\n\n(Information augmented by Google Search)"
            except Exception as e:
                print(f"Error synthesizing answer from search results: {e}")
                final_answer = ("I'm sorry, I encountered an issue while trying to find information. "
                                "Please try again later or contact customer support.")

    return {"response": final_answer}


# --- Simple HTML Template (templates/index.html) ---
# Create a 'templates' directory in the same location as your main.py (or app.py)
# and put this content into index.html:

"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>E-commerce Chatbot</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f8fafc;
        }
        .chat-message {
            max-width: 80%;
            padding: 10px 15px;
            border-radius: 20px;
            margin-bottom: 10px;
            word-wrap: break-word;
        }
        .user-message {
            background-color: #3b82f6; /* blue-500 */
            color: white;
            align-self: flex-end;
            border-bottom-right-radius: 5px;
        }
        .bot-message {
            background-color: #e2e8f0; /* slate-200 */
            color: #1e293b; /* slate-900 */
            align-self: flex-start;
            border-bottom-left-radius: 5px;
        }
        .input-area {
            background-color: white;
            border-top: 1px solid #e2e8f0;
            padding: 1rem;
        }
    </style>
</head>
<body class="flex flex-col h-screen">
    <header class="bg-gradient-to-r from-blue-600 to-indigo-700 text-white p-4 shadow-md rounded-b-lg">
        <h1 class="text-3xl font-bold text-center">🛍️ E-commerce Product Assistant 💬</h1>
        <p class="text-center text-sm mt-1 opacity-90">Ask me anything about our products!</p>
    </header>

    <main class="flex-1 overflow-y-auto p-6 flex flex-col items-center">
        <div id="chat-box" class="w-full max-w-2xl flex flex-col space-y-4">
            <div class="chat-message bot-message">Hello! How can I help you with our products today?</div>
        </div>
    </main>

    <footer class="input-area flex items-center justify-center p-4">
        <div class="w-full max-w-2xl flex rounded-full shadow-lg overflow-hidden">
            <input type="text" id="user-input" placeholder="Type your question here..."
                   class="flex-1 p-4 border-none focus:outline-none rounded-l-full text-lg">
            <button id="send-button"
                    class="bg-blue-600 hover:bg-blue-700 text-white font-semibold py-4 px-6 rounded-r-full
                           transition duration-300 ease-in-out transform hover:scale-105 flex items-center justify-center">
                <svg xmlns="http://www.w3.org/2000/svg" class="h-6 w-6" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                    <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M14 5l7 7m0 0l-7 7m7-7H3" />
                </svg>
            </button>
        </div>
    </footer>

    <script>
        const chatBox = document.getElementById('chat-box');
        const userInput = document.getElementById('user-input');
        const sendButton = document.getElementById('send-button');

        function addMessage(text, sender) {
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('chat-message');
            if (sender === 'user') {
                messageDiv.classList.add('user-message', 'self-end');
            } else {
                messageDiv.classList.add('bot-message', 'self-start');
            }
            messageDiv.textContent = text;
            chatBox.appendChild(messageDiv);
            chatBox.scrollTop = chatBox.scrollHeight; // Scroll to bottom
        }

        async function sendMessage() {
            const query = userInput.value.trim();
            if (!query) return;

            addMessage(query, 'user');
            userInput.value = ''; // Clear input

            addMessage("Thinking...", 'bot'); // Temporary thinking message

            try {
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ query: query }),
                });

                const data = await response.json();
                // Remove "Thinking..." message
                chatBox.removeChild(chatBox.lastChild);
                addMessage(data.response, 'bot');
            } catch (error) {
                console.error('Error:', error);
                chatBox.removeChild(chatBox.lastChild); // Remove thinking message
                addMessage("Oops! Something went wrong. Please try again.", 'bot');
            }
        }

        sendButton.addEventListener('click', sendMessage);
        userInput.addEventListener('keypress', (event) => {
            if (event.key === 'Enter') {
                sendMessage();
            }
        });
    </script>
</body>
</html>
"""
